In [25]:
import json
import csv
from deepdiff import DeepDiff
from pprint import pprint


stg_data_path = "../temp_file/stg_product.jsons"
prod_data_path = "../temp_file/prod_product.jsons"

def convert_json(file_path):
    with open (file_path, "r") as myfile:
        data=myfile.read().replace("}\n{", "},\n{")
        data = "[" + data + "]"
        data = json.loads(data)
        return data

# write missing products to csv
def write_missing_products(pid_list, output_filename):  
    with open(output_filename, "w") as output:
        writer = csv.writer(output, lineterminator='\n')
        for val in pid_list:
            writer.writerow([val])
    
def compare_products(stg_data, prod_data):
    print("Comparing Data")
    comparison_data_dictionary = {}
    missing_pids = []
#   use production data as baseline compare
    for prod_product in prod_data:
        match_found_bool = False
        prod_product_id = prod_product['externalID']       
        for stg_product in stg_data:       
            stg_product_id = stg_product['externalID']        
            if stg_product_id == prod_product_id:
                match_found_bool = True
                ddiff = DeepDiff(prod_product, stg_product, ignore_order=True)
                if bool(ddiff):
#                     print(prod_product_id)
#                     pprint(ddiff)
                    comparison_data_dictionary[prod_product_id] = ddiff
        if not match_found_bool:
#           existing Production product not found in stg_data
            missing_pids.append(prod_product_id)
    write_missing_products(missing_pids, "differ_missing_pids.csv")
    if bool(comparison_data_dictionary):
        pprint(comparison_data_dictionary)
        with open('compare_results.json', 'w') as fp:
            json.dump(comparison_data_dictionary, fp, indent=4)




def main():
    new_stg_data = convert_json(stg_data_path)
    new_prod_data = convert_json(prod_data_path)
    
    compare_products(new_stg_data, new_prod_data)
    
main()

Comparing Data
{'1047237787684': {'values_changed': {"root['attributes']['name']['default']": {'new_value': 'RED '
                                                                                             'GREEN '
                                                                                             'BLUE',
                                                                                'old_value': 'ROLY '
                                                                                             'POLY'},
                                      "root['attributes']['name']['en_US']": {'new_value': 'RED '
                                                                                           'GREEN '
                                                                                           'BLUE',
                                                                              'old_value': 'ROLY '
                                                                                   

TypeError: the JSON object must be str, bytes or bytearray, not dict